In [1]:
!pip install -q -U kaggle_environments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.1/134.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.1 MB/s eta 0:00:00


# **Импорт библиотек**

In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

# **Опишем поведение агентов**

In [20]:
# Агент, который всегда играет "камень"
%%writefile rock_agent.py
def your_agent(observation, configuration):
    return 0  # 0 - камень

Overwriting rock_agent.py


In [8]:
# Агент, который всегда играет "бумагу"
%%writefile paper_agent.py
def your_agent(observation, configuration):
    return 1  # 1 - бумага

Writing paper_agent.py


In [9]:
# Агент, который всегда играет "ножницы"
%%writefile scissors_agent.py
def your_agent(observation, configuration):
    return 2  # 2 - ножницы

Writing scissors_agent.py


In [10]:
# Агент, который повторяет последнее действие противника
%%writefile copy_opponent.py
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction  # Повторяет действие противника
    else:
        return random.choice([0, 1, 2])  # Первое действие случайное

Overwriting copy_opponent.py


In [11]:
# Агент, который играет случайно
%%writefile random_agent.py
def random_agent(observation, configuration):
    return random.choice([0, 1, 2])  # Случайный выбор между камнем, бумагой и ножницами

Writing random_agent.py


In [12]:
# Агент, который меняет действие каждую игру
%%writefile alternating_agent.py
def alternating_agent(observation, configuration):
    return observation.step % 3  # Меняет действия по циклу (камень -> бумага -> ножницы)

Writing alternating_agent.py


In [13]:
# Агент, который играет "камень", если противник играет "ножницы"
%%writefile rock_if_opponent_scissors.py
def rock_if_opponent_scissors(observation, configuration):
    if observation.step > 0 and observation.lastOpponentAction == 2:
        return 0  # Если противник играл ножницы, играем камень
    return random.choice([1, 2])  # В противном случае, случайный выбор

Writing rock_if_opponent_scissors.py


In [14]:
# Агент, который играет "ножницы", если противник играет "бумагу"
%%writefile scissors_if_opponent_paper.py
def scissors_if_opponent_paper(observation, configuration):
    if observation.step > 0 and observation.lastOpponentAction == 1:
        return 2  # Если противник играл бумагу, играем ножницы
    return random.choice([0, 1])  # В противном случае, случайный выбор

Writing scissors_if_opponent_paper.py


In [15]:
# Агент, который играет "бумагу", если противник играет "камень"
%%writefile paper_if_opponent_rock.py
def paper_if_opponent_rock(observation, configuration):
    if observation.step > 0 and observation.lastOpponentAction == 0:
        return 1  # Если противник играл камень, играем бумагу
    return random.choice([0, 2])  # В противном случае, случайный выбор

Writing paper_if_opponent_rock.py


In [16]:
# Агент, который играет "камень" 70% времени, а "бумагу" 30%
%%writefile weighted_rock_agent.py
def weighted_rock_agent(observation, configuration):
    if random.random() < 0.7:
        return 0  # 70% - камень
    return 1  # 30% - бумага

Writing weighted_rock_agent.py


In [17]:
# Агент, который выбирает действия в зависимости от предыдущих ходов
%%writefile pattern_agent.py
history = []

def pattern_agent(observation, configuration):
    global history
    if len(history) < 2:
        action = random.choice([0, 1, 2])  # Случайный выбор, если нет истории
    else:
        # Если последние два хода противника были одинаковыми, играем тем, что его бьет
        if history[-1] == history[-2]:
            action = (history[-1] + 1) % 3  # Бьем то, что он играл
        else:
            action = random.choice([0, 1, 2])  # В противном случае, случайный выбор
    history.append(observation.lastOpponentAction)
    return action

Writing pattern_agent.py


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Агент, который анализирует ход противника и пытается предугадать
%%writefile predictive_agent.py
def predictive_agent(observation, configuration):
    if observation.step == 0:
        return random.choice([0, 1, 2])  # Случайный выбор на первом ходу
    else:
        # Если противник играл "камень" последний раз, можно предположить, что он может сыграть "бумагу"
        if observation.lastOpponentAction == 0:
            return 2  # Играем ножницы
        elif observation.lastOpponentAction == 1:
            return 0  # Играем камень
        else:
            return 1  # Играем бумагу

Writing predictive_agent.py


# **Результаты тестов**

In [19]:
# Тестирование агентов
results = []
agents = [
    "rock_agent.py",
    "paper_agent.py",
    "scissors_agent.py",
    "copy_opponent.py",
    "random_agent.py",
    "alternating_agent.py",
    "rock_if_opponent_scissors.py",
    "scissors_if_opponent_paper.py",
    "paper_if_opponent_rock.py",
    "weighted_rock_agent.py",
    "pattern_agent.py",
    "predictive_agent.py",
]

# Запускаем тесты для каждой пары агентов
for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        res = evaluate(
            "rps",
            [agents[i], agents[j]],
            configuration={"episodeSteps": 100}
        )
        results.append((agents[i], agents[j], res))

# Выводим результаты
for agent1, agent2, res in results:
    print(f"Результаты для {agent1} и {agent2}: {res}")

Результаты для rock_agent.py и paper_agent.py: [[-99.0, 99.0]]
Результаты для rock_agent.py и scissors_agent.py: [[99.0, -99.0]]
Результаты для rock_agent.py и copy_opponent.py: [[1, None]]
Результаты для rock_agent.py и random_agent.py: [[1, None]]
Результаты для rock_agent.py и alternating_agent.py: [[0, 0]]
Результаты для rock_agent.py и rock_if_opponent_scissors.py: [[1, None]]
Результаты для rock_agent.py и scissors_if_opponent_paper.py: [[1, None]]
Результаты для rock_agent.py и paper_if_opponent_rock.py: [[1, None]]
Результаты для rock_agent.py и weighted_rock_agent.py: [[1, None]]
Результаты для rock_agent.py и pattern_agent.py: [[1, None]]
Результаты для rock_agent.py и predictive_agent.py: [[1, None]]
Результаты для paper_agent.py и scissors_agent.py: [[-99.0, 99.0]]
Результаты для paper_agent.py и copy_opponent.py: [[1, None]]
Результаты для paper_agent.py и random_agent.py: [[1, None]]
Результаты для paper_agent.py и alternating_agent.py: [[0, 0]]
Результаты для paper_agent

# **Анализ результатов**

# Анализ результатов

## Общие выводы
- Результаты в формате `[[score1, score2]]` показывают, как каждый из агентов выступил в игре.
- Значение `99.0` означает, что агент выиграл, `-99.0` — что агент проиграл, а `0` — что была ничья.
- Значение `None` может означать, что один из агентов не выполнил свое действие или не был протестирован должным образом.

## Параметры агентов
- **rock_agent.py** и **paper_agent.py**: Их взаимодействие показывает, что бумага всегда побеждает камень (`[-99.0, 99.0]`).
- **rock_agent.py** и **scissors_agent.py**: Камень всегда побеждает ножницы (`[99.0, -99.0]`).
- **paper_agent.py** и **scissors_agent.py**: Здесь также заметно, что бумага побеждает ножницы (`[-99.0, 99.0]`).

## Агенты, которые копируют действия противника
- **copy_opponent.py** взаимодействует с различными агентами, но, как видно из результатов, многие игры заканчиваются `None`, что указывает на отсутствие результата. Можно сделать  вывод, что агент не смог корректно реагировать на действия соперника.

## Случайные агенты
- **random_agent.py** взаимодействует с многими другими агентами, и результаты в большинстве случаев показывают `None`. Можно сделать  вывод, что случайный агент не демонстрирует стабильных результатов.

## Агенты с предсказаниями
- **predictive_agent.py** и другие агенты, такие как **weighted_rock_agent.py** и **pattern_agent.py**, показывают много значений `None`, что может указывать на недостаток в логике или стратегии, применяемой в этих агентах.

## Агенты с адаптивным поведением
- Агенты, которые адаптируются к действиям соперника, такие как **alternating_agent.py**, демонстрируют более положительные результаты, выигрывая в большинстве взаимодействий.